<a href="https://colab.research.google.com/github/ahoucbvtw/NewsClassification-Chinese/blob/main/NewsClassification-Chinese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
path = "drive/My Drive/News/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall("train")
path = "drive/My Drive/News/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall("test")

In [2]:
# "train/chinese_news_trans/*"
import os
import pandas as pd
import glob
contents = []
types = []
def showdata(path):
    for dirName in glob.glob(os.path.join(path, "*")):
        t = os.path.split(dirName)[-1]
        for fn in glob.glob(os.path.join(dirName, "*.txt")) + glob.glob(os.path.join(dirName, "*.TXT")):
            with open(fn) as f:
                contents.append(f.read())
                types.append(t)
    data = pd.DataFrame({"content": contents, "answer": types})
    return data

In [7]:
train_data = showdata("train/chinese_news_trans")
test_data = showdata("test/chinese_news_test")

train_data
# test_data
# print(test_df)

,content,answer
0,日月光華 -- SE精華區文章閱讀\n---------------------------...,計算機
1,日月光華 -- Software精華區文章閱讀\n---------------------...,計算機
2,日月光華 -- Hardware精華區文章閱讀\n---------------------...,計算機
3,日月光華 -- Hardware精華區文章閱讀\n---------------------...,計算機
4,日月光華 -- SE精華區文章閱讀\n---------------------------...,計算機
...,...,...
10846,膠濟鐵路複線二期工程正加緊施工\n新華社濟南５月１４日電（通訊員王建國...,交通
10847,【 日 期 】19960612\n【 版 號 】9\n【 標 題 】認真實施分道航行規...,交通
10848,"\n ★★ 新華社電訊稿數據庫 檢索命中 44375 篇, 顯示第 3104 篇。...",交通
10849,濟南鐵路局積極做好防洪準備工作\n新華社濟南５月２６日電（通訊員孔鑑、...,交通


In [8]:
# 將dataset中的答案統一區分並列出
newsType = train_data["answer"].unique()
newsType_dict = { n:i for i,n in enumerate(newsType)}
newsType_r_dict = { i:n for i,n in enumerate(newsType_dict)}
newsType_dict

{'交通': 9,
 '政治': 8,
 '教育': 7,
 '環境': 1,
 '經濟': 6,
 '藝術': 3,
 '計算機': 0,
 '軍事': 4,
 '醫藥': 2,
 '體育': 5}

In [9]:
#將answer轉換成0123...
train_data["answer"] = train_data["answer"].replace(newsType_dict)
test_data["answer"] = test_data["answer"].replace(newsType_dict)

train_data

,content,answer
0,日月光華 -- SE精華區文章閱讀\n---------------------------...,0
1,日月光華 -- Software精華區文章閱讀\n---------------------...,0
2,日月光華 -- Hardware精華區文章閱讀\n---------------------...,0
3,日月光華 -- Hardware精華區文章閱讀\n---------------------...,0
4,日月光華 -- SE精華區文章閱讀\n---------------------------...,0
...,...,...
10846,膠濟鐵路複線二期工程正加緊施工\n新華社濟南５月１４日電（通訊員王建國...,9
10847,【 日 期 】19960612\n【 版 號 】9\n【 標 題 】認真實施分道航行規...,9
10848,"\n ★★ 新華社電訊稿數據庫 檢索命中 44375 篇, 顯示第 3104 篇。...",9
10849,濟南鐵路局積極做好防洪準備工作\n新華社濟南５月２６日電（通訊員孔鑑、...,9


In [10]:
# 新創一Function將新聞內部的換行符號刪除並與之前資料表結合
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def newscut(news):
    news = news.replace("\r", "").replace("\n", "").replace("\n\n","")
    return " ".join(jieba.cut(news))

train_data["content"] = train_data["content"].apply(newscut)
test_data["content"] = test_data["content"].apply(newscut)
train_data

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.813 seconds.
Prefix dict has been built successfully.


,content,answer
0,日月 光華 -- SE 精華區 文章 閱讀 --------------------...,0
1,日月 光華 -- Software 精華區 文章 閱讀 --------------...,0
2,日月 光華 -- Hardware 精華區 文章 閱讀 --------------...,0
3,日月 光華 -- Hardware 精華區 文章 閱讀 --------------...,0
4,日月 光華 -- SE 精華區 文章 閱讀 --------------------...,0
...,...,...
10846,膠濟 鐵路 複線 二期工程 正 加緊 施工 新華...,9
10847,【 日 期 】 19960612 【 版 號 】 9 【 ...,9
10848,★ ★ 新華社 電訊稿 數據庫 檢索 命中 44375 ...,9
10849,濟南鐵路局 積極 做好 防洪 準備 工作 新華社...,9


In [11]:
# countvectorizer
# 訓練資料=> fit(統計有多少分詞標籤)/transform(統計這些分詞標籤出現次數),所以訓練資料2個都要 = fit_transform
# 測試資料=> transform(訓練資料不須再做分詞標籤，因為只須透過訓練分詞來分辨判斷，新的分詞就不理)
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
train_count =  vec.fit_transform(train_data["content"])
test_count = vec.transform(test_data["content"])
#訓練資料裡每首詩對每個分詞欄位出現的次數
print(train_count)
# vec.vocabulary_ => 字典
# 利用vec.vocabulary_[","]等等標點符號或換行是否還在裡面
#vec.vocabulary_ (每個train_count中分詞欄位對應的分詞文字)

#測試資料裡每首詩對每個分詞欄位出現的次數
print(test_count)

  (0, 40438)	4
  (0, 14926)	4
  (0, 5830)	3
  (0, 56925)	1
  (0, 39637)	1
  (0, 72379)	1
  (0, 52712)	1
  (0, 4362)	4
  (0, 52084)	2
  (0, 68229)	2
  (0, 64881)	7
  (0, 52713)	1
  (0, 849)	1
  (0, 26)	2
  (0, 61)	1
  (0, 218)	1
  (0, 1527)	1
  (0, 0)	1
  (0, 40744)	1
  (0, 13463)	1
  (0, 11970)	1
  (0, 39556)	1
  (0, 68230)	1
  (0, 64721)	2
  (0, 20168)	1
  :	:
  (10850, 16012)	2
  (10850, 47769)	4
  (10850, 11664)	1
  (10850, 64016)	1
  (10850, 47767)	2
  (10850, 21486)	1
  (10850, 49691)	3
  (10850, 26259)	4
  (10850, 69153)	3
  (10850, 54623)	2
  (10850, 67695)	1
  (10850, 73136)	1
  (10850, 58602)	1
  (10850, 44431)	1
  (10850, 69249)	1
  (10850, 39715)	1
  (10850, 76804)	3
  (10850, 29061)	1
  (10850, 73572)	1
  (10850, 67401)	1
  (10850, 19017)	1
  (10850, 65800)	1
  (10850, 43567)	1
  (10850, 32976)	1
  (10850, 60258)	1
  (0, 0)	1
  (0, 20)	1
  (0, 26)	2
  (0, 61)	1
  (0, 141)	1
  (0, 218)	1
  (0, 464)	1
  (0, 832)	1
  (0, 849)	1
  (0, 873)	1
  (0, 937)	1
  (0, 1457)	1
  (0, 152

In [12]:
# 3種單純貝氏'
# Gaussian: 連續特徵=>高斯分佈(常態分佈:可以少少的資料就做出一條全部符合此常態方程式,ex:x=4.1、4.11、4.111...都可以透過此常態方程式算出各個機率)
# Muitinomial: 整數特徵(文字)
# Beronull: 只有「是否」2種特徵
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_count, train_data["answer"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
#利用測試資料驗證計算出來的模型公式好壞
from sklearn.metrics import accuracy_score
pre = clf.predict(test_count)
print("預測結果:", list(pre))
print("正確結果:", list(test_data["answer"]))
print("正確率:", accuracy_score(pre, test_data["answer"]))

預測結果: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(test_data["answer"], pre), columns = [n + ("predict") for n in newsType], index = [n + ("true") for n in newsType])

,計算機predict,環境predict,醫藥predict,藝術predict,軍事predict,體育predict,經濟predict,教育predict,政治predict,交通predict
計算機true,728,0,0,0,0,0,16,0,0,0
環境true,0,764,0,0,0,0,0,0,4,0
醫藥true,0,0,804,0,0,0,0,4,8,0
藝術true,0,0,0,964,0,0,0,0,4,0
軍事true,0,0,0,0,840,0,0,12,108,0
體育true,0,0,0,0,0,1792,0,0,4,0
經濟true,0,0,0,4,0,0,1220,0,16,0
教育true,0,0,0,0,0,0,0,860,4,0
政治true,0,0,0,0,4,0,0,0,2016,0
交通true,0,0,0,0,0,0,0,0,0,776


In [15]:
#利用使用者輸入新聞內容來預測是屬於哪類文章? 環境、經濟...
s = input("輸入新聞內容:")
#當要predict時一定要先轉群集(list)
inputs = [newscut(s)]
x = vec.transform(inputs)
#依照使用者輸入的預測為屬於哪類文章
probality = clf.predict_proba(x)[0]
print("所有類型文章的預測機率:", probality)
print("使用者輸入:", inputs)

for name, prob in zip(newsType, probality):
    print(name, "的預測機率:", round(prob, 2))
userpre = clf.predict(x)[0]
# argmax: 找最大值的index
print(f"預測結果此文章類型為『{newsType_r_dict[probality.argmax()]}』最高！！")

輸入新聞內容:中央流行疫情指揮中心13日持續因應新冠肺炎（武漢肺炎）召開記者會，指揮官陳時中宣布新增25例確診個案，其中12例為境外移入個案、13例本土個案。指揮中心12日公布案1203相關接觸者有10人確診，包括案1218、案1219，及案1223至案1230等10人為5女5男，若加上13日新增的9名個案，該群聚感染案已達20人。  指揮中心表示，案1245、案1246、案1248、案1250、案1251、案1253及案1255至案1257等9人，為7男2女，年齡介於30多歲至80多歲，其中4人無症狀，5人曾有症狀（發病日介於6日至11日）。9人因曾與案1203於同餐廳用餐，匡列為接觸者並進行採檢，於13日確診；截至目前案1203相關群聚共計20人確診。
所有類型文章的預測機率: [5.77023079e-17 6.29196989e-12 1.00000000e+00 2.37279733e-23
 2.62898251e-10 8.05683707e-20 1.19278161e-12 4.33696480e-28
 1.60223389e-56 5.52058379e-12]
使用者輸入: ['中央 流行 疫情 指揮中心 13 日 持續 因應 新冠 肺炎 （ 武漢 肺炎 ） 召開 記者會 ， 指揮官 陳 時 中 宣布 新增 25 例 確診 個案 ， 其中 12 例為 境外 移入 個案 、 13 例 本土 個案 。 指揮中心 12 日 公布 案 1203 相關 接觸 者 有 10 人 確診 ， 包括 案 1218 、 案 1219 ， 及案 1223 至案 1230 等 10 人為 5 女 5 男 ， 若 加上 13 日 新增 的 9 名 個案 ， 該 群聚 感染 案 已 達 20 人 。     指揮中心 表示 ， 案 1245 、 案 1246 、 案 1248 、 案 1250 、 案 1251 、 案 1253 及案 1255 至案 1257 等 9 人 ， 為 7 男 2 女 ， 年齡 介於 30 多歲 至 80 多歲 ， 其中 4 人 無症狀 ， 5 人 曾 有 症狀 （ 發病 日 介於 6 日至 11 日 ） 。 9 人因 曾 與 案 1203 於 同 餐廳 用餐 ， 匡列 為 接觸 者並 進行 採檢 ， 於 13 日 確診 ； 截至 目前 案 